# Download and build the cell

We start with searching the [Allen Cell Types Database](http://celltypes.brain-map.org/data) for a detailed biophysical cell model that could faithfully represent the provided biological data. This step can be skipped if the cell model was already obtained from a different source.

## Find and download the cell

We found that [the cell from specimen 480351780](https://celltypes.brain-map.org/experiment/electrophysiology/480351780) has a biophysical model with active channels in the dendrites that was constructed to represent data from the same general area of the mouse brain. To download the model, we first use `specimen_id` to view all models available for that specimen.

In [ ]:
from allensdk.api.queries.biophysical_api import BiophysicalApi

bp = BiophysicalApi()
bp.get_neuronal_models(480351780)

# Note
# ----
# perisomatic – passive dendrites
# all active - active dendrites

[{'id': 482529696,
  'name': 'Biophysical - perisomatic_Rorb-IRES2-Cre-D;Ai14-197851.04.01.01',
  'neuron_reconstruction_id': 491770985,
  'neuronal_model_template_id': 329230710,
  'specimen_id': 480351780},
 {'id': 497232429,
  'name': 'Biophysical - all active_Rorb-IRES2-Cre-D;Ai14-197851.04.01.01',
  'neuron_reconstruction_id': 491770985,
  'neuronal_model_template_id': 491455321,
  'specimen_id': 480351780}]

We then use cell id (`id`) to download the cell data to `working_directory`.

In [ ]:
# Download the cell
bp.cache_stimulus = False # False = do not download the large stimulus NWB file
bp.cache_data(497232429, working_directory = 'L5') # The cell will be downloaded to `working_directory`

2025-07-04 17:14:26,944 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/491770987
2025-07-04 17:14:27,425 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/496605092
2025-07-04 17:14:27,654 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337019
2025-07-04 17:14:27,862 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337225
2025-07-04 17:14:28,049 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337054
2025-07-04 17:14:28,251 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/395337070
2025-07-04 17:14:28,431 allensdk.api.api.retrieve_fi

## Build the cell

`ACT` can flexibly build custom cell models, including those downloaded from ACTD, by processing a cell builder function provided by the user (see [an example](https://github.com/V-Marco/ACT/blob/main/experiments/L5/cell_builder.py) of specifying such a function for the downloaded cell).

``` python
def L5_cell_builder():
    from allensdk.model.biophys_sim.config import Config
    from allensdk.model.biophysical.utils import AllActiveUtils
    import os

    # Create the h object
    current_dir = os.getcwd()
    os.chdir("../../data/L5/orig")
    description = Config().load('manifest.json')
    utils = AllActiveUtils(description, axon_type = "stub")
    h = utils.h

    # Configure morphology
    morphology_path = description.manifest.get_path('MORPHOLOGY')
    utils.generate_morphology(morphology_path.encode('ascii', 'ignore'))
    utils.load_cell_parameters()
    os.chdir(current_dir)

    return h
```

## Additional resources by Allen Institute

The following resources could be useful:

- [Tutorial on downloading biophysical models](https://allensdk.readthedocs.io/en/latest/biophysical_models.html)
- [Example on building cell models](https://alleninstitute.github.io/AllenSDK/_static/examples/nb/pulse_stimulus.html)